In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv('dirty_cafe_sales.csv')

In [3]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [5]:
df.shape

(10000, 8)

In [6]:
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [7]:
missing_percentage = (df.isna().sum()/len(df))*100
print(missing_percentage)

Transaction ID       0.00
Item                 3.33
Quantity             1.38
Price Per Unit       1.79
Total Spent          1.73
Payment Method      25.79
Location            32.65
Transaction Date     1.59
dtype: float64


In [8]:
df['Payment Method'] = df['Payment Method'].replace("UNKNOWN", np.nan)
df['Payment Method'] = df['Payment Method'].replace("ERROR", np.nan)
df['Payment Method'] = df['Payment Method'].fillna('Unknown')


In [9]:
df['Location'] = df['Location'].replace("UNKNOWN", np.nan)
df['Location'] = df['Location'].replace("ERROR", np.nan)
df['Location'] = df['Location'].fillna('Unknown')

In [10]:
missing_percentage = (df.isna().sum()/len(df))*100
print(missing_percentage)

Transaction ID      0.00
Item                3.33
Quantity            1.38
Price Per Unit      1.79
Total Spent         1.73
Payment Method      0.00
Location            0.00
Transaction Date    1.59
dtype: float64


In [11]:
# Cleaning column - Price Per Unit

df['Price Per Unit'] = df['Price Per Unit'].replace('UNKNOWN', np.nan)
df['Price Per Unit'] = df['Price Per Unit'].replace('ERROR', np.nan)
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')

In [12]:
itemPrice = df.groupby('Item')['Price Per Unit'].mean().to_dict()
print(itemPrice)

{'Cake': 3.0, 'Coffee': 2.0, 'Cookie': 1.0, 'ERROR': 2.9444444444444446, 'Juice': 3.0, 'Salad': 5.0, 'Sandwich': 4.0, 'Smoothie': 4.0, 'Tea': 1.5, 'UNKNOWN': 2.8803680981595092}


In [13]:
df['Price Per Unit'] = df.apply(
    lambda row: itemPrice.get(row['Item'], row['Price Per Unit']) if pd.isna(row['Price Per Unit']) else row['Price Per Unit'],
    axis=1
)

df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Price Per Unit'].mean())

In [14]:
df['Price Per Unit'].isna().sum()

0

In [15]:
# Cleaning column : Price
df['Item'].isna().sum()

333

In [16]:
df['Item'] = df['Item'].replace('UNKNOWN', np.nan)
df['Item'] = df['Item'].replace('ERROR', np.nan)

In [17]:
price_to_item = df.groupby('Price Per Unit')['Item'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan).to_dict()
print(price_to_item)

{1.0: 'Cookie', 1.5: 'Tea', 2.0: 'Coffee', 2.8803680981595092: nan, 2.9444444444444446: nan, 2.9475417864633306: nan, 3.0: 'Juice', 4.0: 'Sandwich', 5.0: 'Salad'}


In [18]:
df['Item'] = df.apply(
    lambda row: price_to_item.get(row['Price Per Unit'], row['Item']) if pd.isna(row['Item']) else row['Item'],
    axis=1
)

In [19]:
df['Item'] = df['Item'].fillna(df['Item'].mode()[0])

df['Item'].isna().sum()

0

In [20]:
df['Item'].value_counts()

Item
Juice       1459
Sandwich    1344
Coffee      1284
Salad       1270
Cookie      1209
Tea         1199
Cake        1139
Smoothie    1096
Name: count, dtype: int64

In [21]:
#Clean column Quantity
df['Quantity'] = df['Quantity'].replace('ERROR', np.nan)
df['Quantity'] = df['Quantity'].replace('UNKNOWN', np.nan)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')
print(df['Quantity'].isna().sum())


479


In [22]:

df['Quantity'] = df.apply(
    lambda row: row['Total Spent'] / row['Price Per Unit']
    if pd.isna(row['Quantity']) and pd.notna(row['Total Spent']) and pd.notna(row['Price Per Unit']) and row['Price Per Unit'] != 0
    else row['Quantity'],
    axis=1
)
# df[df['Quantity'] == 'ERROR']
df['Quantity'] = df['Quantity'].round()

df['Quantity'] = df['Quantity'].fillna(1).astype(int)


In [23]:
df['Quantity'].value_counts()

Quantity
5    2108
2    2056
3    1947
1    1946
4    1941
9       1
7       1
Name: count, dtype: int64

In [24]:
df[df['Total Spent'].isna()]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
2,TXN_4271903,Cookie,4,1.0,NaN,Credit Card,In-store,2023-07-19
25,TXN_7958992,Smoothie,3,4.0,NaN,Unknown,Unknown,2023-12-13
31,TXN_8927252,Cookie,2,1.0,NaN,Credit Card,Unknown,2023-11-06
42,TXN_6650263,Tea,2,1.5,NaN,Unknown,Takeaway,2023-01-10
65,TXN_4987129,Sandwich,3,4.0,NaN,Unknown,In-store,2023-10-20
...,...,...,...,...,...,...,...,...
9893,TXN_3809533,Juice,2,3.0,NaN,Digital Wallet,Takeaway,2023-02-02
9954,TXN_1191659,Coffee,4,2.0,NaN,Credit Card,In-store,2023-11-21
9977,TXN_5548914,Juice,2,3.0,NaN,Digital Wallet,In-store,2023-11-04
9988,TXN_9594133,Cake,5,3.0,NaN,Unknown,Unknown,NaN


In [25]:
df['Total Spent'] = df.apply(
    lambda row: row['Quantity'] * row['Price Per Unit']
    if pd.isna(row['Total Spent']) and pd.notna(row['Quantity']) and pd.notna(row['Price Per Unit'])
    else row['Total Spent'],
    axis=1
)

In [26]:
df['Total Spent'] = df['Total Spent'].round()
df['Total Spent'].value_counts()

Total Spent
4.0     1213
6.0     1020
12.0     998
3.0      972
8.0      970
20.0     789
15.0     766
2.0      736
10.0     542
9.0      510
5.0      498
16.0     466
25.0     269
1.0      251
Name: count, dtype: int64

In [27]:
df['Payment Method'].value_counts(dropna=False)


Payment Method
Unknown           3178
Digital Wallet    2291
Credit Card       2273
Cash              2258
Name: count, dtype: int64

In [28]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')
df['Transaction Date'] = df['Transaction Date'].fillna(df['Transaction Date'].mode()[0])


In [29]:
df.isna().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

In [30]:
df.shape

(10000, 8)

In [31]:
df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,Unknown,2023-03-31
6,TXN_4433211,Juice,3,3.0,9.0,Unknown,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,Unknown,2023-10-28
8,TXN_4717867,Juice,5,3.0,15.0,Unknown,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,Unknown,In-store,2023-12-31


In [35]:
df.to_csv('cleaned_transactions.csv', index=False)